<a href="https://colab.research.google.com/github/seecode4/seeRepo1/blob/main/mec2-projects/Student_MLE_MiniProject_DL_HyperParamTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q scikeras

import pandas as pd
import numpy as np
import time

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.metrics import RocCurveDisplay

from keras.models import Sequential
from keras.layers import Dense
from keras.initializers import RandomNormal, RandomUniform
from scikeras.wrappers import KerasClassifier
from sklearn.pipeline import Pipeline


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 551.0 kB/s eta 0:00:00


In [3]:
DATA_PATH = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'

# Download the dataset and load it into a pandas DataFrame
df_data = pd.read_csv(DATA_PATH, header=None)

In [4]:
# Display the first few rows of the DataFrame
print(df_data.head())
print(df_data.shape)

   0                  1       2           3   4                    5   \
0  39          State-gov   77516   Bachelors  13        Never-married   
1  50   Self-emp-not-inc   83311   Bachelors  13   Married-civ-spouse   
2  38            Private  215646     HS-grad   9             Divorced   
3  53            Private  234721        11th   7   Married-civ-spouse   
4  28            Private  338409   Bachelors  13   Married-civ-spouse   

                   6               7       8        9     10  11  12  \
0        Adm-clerical   Not-in-family   White     Male  2174   0  40   
1     Exec-managerial         Husband   White     Male     0   0  13   
2   Handlers-cleaners   Not-in-family   White     Male     0   0  40   
3   Handlers-cleaners         Husband   Black     Male     0   0  40   
4      Prof-specialty            Wife   Black   Female     0   0  40   

               13      14  
0   United-States   <=50K  
1   United-States   <=50K  
2   United-States   <=50K  
3   United-State

In [5]:
# Ref: https://archive.ics.uci.edu/dataset/2/adult
feat_names = ['age', 'workclass', 'fnlwgt', 'education', 'education-num',
             'marital-status', 'occupation', 'relationship', 'race', 'sex',
             'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
             'income']
# Combine both
# help(df_data.set_axis)
df_raw = df_data.set_axis(feat_names, axis='columns')
print(df_raw.head())

   age          workclass  fnlwgt   education  education-num  \
0   39          State-gov   77516   Bachelors             13   
1   50   Self-emp-not-inc   83311   Bachelors             13   
2   38            Private  215646     HS-grad              9   
3   53            Private  234721        11th              7   
4   28            Private  338409   Bachelors             13   

        marital-status          occupation    relationship    race      sex  \
0        Never-married        Adm-clerical   Not-in-family   White     Male   
1   Married-civ-spouse     Exec-managerial         Husband   White     Male   
2             Divorced   Handlers-cleaners   Not-in-family   White     Male   
3   Married-civ-spouse   Handlers-cleaners         Husband   Black     Male   
4   Married-civ-spouse      Prof-specialty            Wife   Black   Female   

   capital-gain  capital-loss  hours-per-week  native-country  income  
0          2174             0              40   United-States   <=50

In [6]:
# Do some exploratory analysis. How many rows/columns are there? How are NULL
# values represented? What's the percentrage of positive cases in the dataset?
num_rows = df_raw.shape[0]
num_cols = df_raw.shape[1]
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_cols}")
print("Column, i.e, feature names:\n", df_raw.columns[0:-1])
print("Target:", df_raw.columns[-1], "\n")
print("Column Data Types:\n", df_raw.dtypes)
print("\nNumber of isnull:\n", df_raw.isnull().sum())
print("Number of isna:\n", df_raw.isna().sum())
print("\nValue Counts for each feature:")
for featname in df_raw.columns:
  print(f"\n-----{featname}-----\n {df_raw[featname].value_counts()}")
target_value_counts = list(df_raw['income'].value_counts())
income_le_50k_pcnt = 100 * target_value_counts[0]/num_rows
income_gt_50k_pcnt = 100 * target_value_counts[1]/num_rows
print(f"\nPercentage of people with income <= 50K: {income_le_50k_pcnt:0.2f}%")
print(f"Percentage of people with income > 50K: {income_gt_50k_pcnt:0.2f}%")

Number of rows: 32561
Number of columns: 15
Column, i.e, feature names:
 Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country'],
      dtype='object')
Target: income 

Column Data Types:
 age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
income            object
dtype: object

Number of isnull:
 age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-

In [7]:
# Find all NULL values and drop them
# Check if value_counts match for this check
df = df_raw
print("Unknown Occupation Before drop null:", (df.loc[:,"occupation"].str.strip() == '?').sum())
for featname in df.columns:
  # string columns are of type 'object'
  if (df[featname].dtype == 'object'):
    # drop all rows that contain '?' which is the same as NULL here
    df = df.drop(df[df[featname].str.strip() == '?'].index)
print("Unknown Occupation After drop null:", (df.loc[:,"occupation"].str.strip() == '?').sum())

print("df shape after drop null:", df.shape)

Unknown Occupation Before drop null: 1843
Unknown Occupation After drop null: 0
df shape after drop null: (30162, 15)


In [8]:
# Use Scikit-Learn's LabelEncoder to convert the income column with a data type
# string to a binary variable.
le = LabelEncoder()
df['income'] = le.fit_transform(df['income'])
print(df['income'].value_counts())

income
0    22654
1     7508
Name: count, dtype: int64


In [9]:
# Split dataset into training and test sets
# Lst[ Initial : End : IndexJump ]
print(df.columns)
X = df.iloc[:, 0:-1]
y = df.loc[:, 'income']
print(X.shape)
print(y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
print(X_train.shape)
print(X_test.shape)

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')
(30162, 14)
(30162,)
(24129, 14)
(6033, 14)


In [10]:
import tensorflow as tf

note=""" In tensorflow, a random operation relies on two different seeds:
a global seed, set by tf.set_random_seed, and
an operation seed, provided as an argument to the operation.
"""
# Config to make results repeatable/deterministic
tf.config.experimental.enable_op_determinism()
np.random.seed(1)
SEED=43
tf.random.set_seed(SEED)
tf.keras.utils.set_random_seed(SEED)

In [11]:
# Use Scikit-Learn's ColumnTransformer to apply One Hot Encoding to the
# categorical variables in workclass, education, marital-status, occupation,
# relationship, 'race', sex, and native-country. #Also, apply MinMaxScaler to
# the remaining continuous features.
note = """
Each transformer is a three-element tuple that defines the name of the transformer,
 the transform to apply, and the column indices to apply it to.
For example:
(Name, Object, Columns)
transformer = ColumnTransformer(transformers=[('cat', OneHotEncoder(), [0, 1])])
"""
dts = X.dtypes

def get_col_indexes(df, col_names):
  return [df.columns.get_loc(name) for name in col_names]

# Categorical features and corresponding column index
cat_columns = [k for k,v in dts.items() if (v == 'object')]
cat_ix = get_col_indexes(X, cat_columns)

# Continuous features and corresponding column index
cont_columns = [k for k,v in dts.items() if (v != 'object')]
cont_ix = get_col_indexes(X, cont_columns)
print("cat_ix:", cat_ix, " cont_ix:", cont_ix)

# data preparation for categorical and continuous features
t = [('cat', OneHotEncoder(handle_unknown='ignore'), cat_ix),
     ('cont', MinMaxScaler(), cont_ix)]
ct = ColumnTransformer(transformers=t)

cat_ix: [1, 3, 5, 6, 7, 8, 9, 13]  cont_ix: [0, 2, 4, 10, 11, 12]


In [12]:
%%time
# Function to create model, required for KerasClassifier
def create_model(input_dim, num_layers=2, num_neurons=8, act_function='relu'):
  """With input_dim given create keras Sequential model with 3 hidden layers of 8, 16 and 8 neurons"""
  # create model
  initializer = RandomNormal(mean=0.0, stddev=1.0, seed=SEED)
  # initializer = RandomUniform(minval=0.0, maxval=1.0, seed=SEED)
  model = Sequential()
  model.add(tf.keras.Input(shape=(input_dim,)))
  for i in range(num_layers):
      model.add(Dense(num_neurons, activation=act_function,
                      kernel_initializer=initializer,
                      use_bias=False))
  model.add(Dense(1, activation='sigmoid', kernel_initializer=initializer,))
  # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 10 µs


In [13]:
# Basic Model without pipeline

# Column Transformer fit transform on X
X_trans = ct.fit_transform(X)
ct_feat_names = ct.get_feature_names_out()
print(X_trans.shape, type(X_trans))
print("Number of columns after transformation:", ct_feat_names.shape[0])

# Simple keras "Sequential" model with 1 hidden layer
Xt_train, Xt_test, yt_train, yt_test = train_test_split(X_trans, y, test_size=0.25, random_state=1)
print(Xt_train.shape, yt_train.shape)

num_feat = Xt_train.shape[1]
print("Num features in input for model:", type(num_feat), num_feat)

# Define the Keras model
basic_model = create_model(num_feat, num_layers=1, num_neurons=8, act_function='relu')
basic_model.summary()

# Fit to data
act_result = basic_model.fit(Xt_train, yt_train, epochs=10, batch_size=64)

# Test if the model is well assembled by predicting before training
yt_pred = basic_model.predict(Xt_train)
print("y_pred shape", yt_pred.shape, "\n First 3 values", yt_pred[0:3])
make_bin = lambda val: 1 if val > 0.5 else 0
yt_pred_df = pd.DataFrame(np.array([make_bin(yi) for yi in yt_pred]))
print(type(yt_pred_df), yt_pred_df.shape, "\n",
      yt_pred_df.value_counts(), yt_train.value_counts())

# Test, Loss and accuracy
loss_and_metrics = basic_model.evaluate(Xt_test, yt_test)
print('Loss = ',loss_and_metrics[0])
print('Accuracy = ',loss_and_metrics[1])

yt_test_pred = basic_model.predict(Xt_test)
basic_score = roc_auc_score(yt_test, yt_test_pred)
print(f"\nroc_auc_score with basic_model: {basic_score:0.3f}")

(30162, 104) <class 'scipy.sparse._csr.csr_matrix'>
Number of columns after transformation: 104
(22621, 104) (22621,)
Num features in input for model: <class 'int'> 104


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 8)                   │             832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │               9 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 841 (3.29 KB)

 Trainable params: 841 (3.29 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
354/354 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7419 - loss: 0.6281
Epoch 2/10
354/354 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7707 - loss: 0.4849
Epoch 3/10
354/354 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7877 - loss: 0.4368
Epoch 4/10
354/354 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8094 - loss: 0.4029
Epoch 5/10
354/354 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8216 - loss: 0.3829
Epoch 6/10
354/354 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8271 - loss: 0.3717
Epoch 7/10
354/354 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8305 - loss: 0.3647
Epoch 8/10
354/354 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8337 - loss: 0.3600
Epoch 9/10
354/354 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8357 - loss: 0.3564
Epoch 10/10
354/354 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8372 - loss: 0.3536
707/707 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
y_pred shape (22621, 1) 
 First 3 values [[0.5420019 ]
 [0.01367371]
 [0.68324465]]
<class 'pandas

In [28]:
%%time
# define create_model(input_dim, num_layers=2, num_neurons=8, act_function='relu')

# Hpyer parameter Tuning
def create_pipeline(m_input_dim, m_nl, m_nn, m_af):
  # Create KerasClassifier
  kerasclf_model = KerasClassifier(model=create_model, verbose=0,
                                   input_dim=m_input_dim,
                                   num_layers=m_nl,
                                   num_neurons=m_nn,
                                   act_function=m_af)
  # Create the scikit-learn pipeline
  kpipe = Pipeline(steps=[
      ('prep', ct),
       ('clf', kerasclf_model)
       ])
  return kpipe

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 9.3 µs


In [29]:
%%time
import time

# With given hyperparam list fit, predict and get roc_auc_score
def kpipe_fit_pred_score(n_ep, n_bsize, num_pipe_feat, m_nl, m_nn, m_af):
  """Expect X_train, X_test, y_train, y_test already there;
  create_pipeline, fit, predict and get roc_auc_score """
  print(f"Tune with epochs={n_ep}  batch_size={n_bsize}",
        f"nlayers={m_nl}  nunits={m_nn} actfunc={m_af}")
  kpipe = create_pipeline(num_pipe_feat, m_nl, m_nn, m_af)
  start_time = time.time()
  # fit data to pipeline model
  history = kpipe.fit(X_train, y_train,
                      clf__epochs=n_ep, clf__batch_size=n_bsize)
  # time to train
  kclf_exectime = (time.time() - start_time)
  # predict test data
  y_test_pipe_pred = kpipe.predict(X_test)
  # get roc_auc_score
  kclf_score = (roc_auc_score(y_test, y_test_pipe_pred))
  return kclf_score, kclf_exectime

CPU times: user 9 µs, sys: 0 ns, total: 9 µs
Wall time: 12.4 µs


In [39]:
# Try differen values for
# [n_epochs, n_batchsize, num_pipe_feat, m_nlayers, m_nunits, m_actfunc]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
print(X_train.shape, y_train.shape)

hplist1 = [[10, 64, 1, 8, 'relu'],
 [40, 64, 3, 8, 'relu'], [40, 64, 3, 8, 'sigmoid'],
  [80, 64, 2, 16, 'relu'],
  [100, 128, 2, 32, 'sigmoid']]
kclf_scores = []
kclf_exectimes = []
num_pipe_feat = 104
numsets1 = len(hplist1)
print("Number of hyperparam sets:", numsets)
for i in range(numsets1):
  hp = hplist1[i]
  kclf_score, kclf_exectime = kpipe_fit_pred_score(hp[0], hp[1],
                                                   num_pipe_feat,
                                                   hp[2], hp[3], hp[4])
  kclf_scores.append(kclf_score)
  kclf_exectimes.append(kclf_exectime)
  print(f"Model Index: {i}, epochs={hp[0]}  batch_size={hp[1]}",
        f"nlayers={hp[2]}  nunits={hp[3]}  actfunc={hp[4]}",
        f"score={kclf_scores[i]:0.4f} exectime={kclf_exectimes[i]:0.4f}")

(24129, 14) (24129,)
Number of hyperparam sets: 5
Tune with epochs=10  batch_size=64 nlayers=1  nunits=8 actfunc=relu
Model Index: 0, epochs=10  batch_size=64 nlayers=1  nunits=8  actfunc=relu score=0.7638 exectime=17.2720
Tune with epochs=40  batch_size=64 nlayers=3  nunits=8 actfunc=relu
Model Index: 1, epochs=40  batch_size=64 nlayers=3  nunits=8  actfunc=relu score=0.7918 exectime=50.6379
Tune with epochs=40  batch_size=64 nlayers=3  nunits=8 actfunc=sigmoid
Model Index: 2, epochs=40  batch_size=64 nlayers=3  nunits=8  actfunc=sigmoid score=0.7926 exectime=49.6297
Tune with epochs=80  batch_size=64 nlayers=2  nunits=16 actfunc=relu
Model Index: 3, epochs=80  batch_size=64 nlayers=2  nunits=16  actfunc=relu score=0.7911 exectime=90.8407
Tune with epochs=100  batch_size=128 nlayers=2  nunits=32 actfunc=sigmoid
Model Index: 4, epochs=100  batch_size=128 nlayers=2  nunits=32  actfunc=sigmoid score=0.7889 exectime=64.0505


In [40]:
hplist2 = [[24, 64, 2, 4, 'sigmoid'], [24, 128, 2, 8, 'sigmoid'],
           [32, 64, 3, 8, 'relu'], [32, 72, 2, 6, 'relu'],
            [32, 64, 3, 6, 'sigmoid'], [32, 128, 2, 4, 'sigmoid'],
             [40, 72, 3, 8, 'relu']]
num_pipe_feat = 104
numsets2 = len(hplist2)
hplist
print("Number of hyperparam sets:", numsets)
for i in range(numsets2):
  hp = hplist2[i]
  kclf_score, kclf_exectime = kpipe_fit_pred_score(hp[0], hp[1],
                                                   num_pipe_feat,
                                                   hp[2], hp[3], hp[4])
  kclf_scores.append(kclf_score)
  kclf_exectimes.append(kclf_exectime)
  print(f"Model Index: {i+numsets1}, epochs={hp[0]}  batch_size={hp[1]}",
        f"nlayers={hp[2]}  nunits={hp[3]}  actfunc={hp[4]}",
        f"score={kclf_scores[i]:0.4f} exectime={kclf_exectimes[i]:0.4f}")

Number of hyperparam sets: 5
Tune with epochs=24  batch_size=64 nlayers=2  nunits=4 actfunc=sigmoid
Model Index: 5, epochs=24  batch_size=64 nlayers=2  nunits=4  actfunc=sigmoid score=0.7638 exectime=17.2720
Tune with epochs=24  batch_size=128 nlayers=2  nunits=8 actfunc=sigmoid
Model Index: 6, epochs=24  batch_size=128 nlayers=2  nunits=8  actfunc=sigmoid score=0.7918 exectime=50.6379
Tune with epochs=32  batch_size=64 nlayers=3  nunits=8 actfunc=relu
Model Index: 7, epochs=32  batch_size=64 nlayers=3  nunits=8  actfunc=relu score=0.7926 exectime=49.6297
Tune with epochs=32  batch_size=72 nlayers=2  nunits=6 actfunc=relu
Model Index: 8, epochs=32  batch_size=72 nlayers=2  nunits=6  actfunc=relu score=0.7911 exectime=90.8407
Tune with epochs=32  batch_size=64 nlayers=3  nunits=6 actfunc=sigmoid
Model Index: 9, epochs=32  batch_size=64 nlayers=3  nunits=6  actfunc=sigmoid score=0.7889 exectime=64.0505
Tune with epochs=32  batch_size=128 nlayers=2  nunits=4 actfunc=sigmoid
Model Index: 1

In [35]:
# Scores with different tries
for i in range(numsets):
  if (kclf_scores[i] > 0.7):
      print(f"Model Index: {i}, epochs={hplist[i][0]}  batch_size={hplist[i][1]}",
            f"  nlayers={hplist[i][2]}  nunits={hplist[i][3]}  ",
            f"score={kclf_scores[i]:0.4f}  exectime={kclf_exectimes[i]:0.4f}")

Model Index: 0, epochs=10  batch_size=64   nlayers=1  nunits=8   score=0.7638  exectime=13.9630
Model Index: 1, epochs=40  batch_size=64   nlayers=3  nunits=8   score=0.7918  exectime=52.9616
Model Index: 2, epochs=40  batch_size=64   nlayers=3  nunits=8   score=0.7926  exectime=51.8629
Model Index: 3, epochs=80  batch_size=64   nlayers=2  nunits=16   score=0.7911  exectime=94.3462
Model Index: 4, epochs=100  batch_size=128   nlayers=2  nunits=32   score=0.7889  exectime=67.1134


## Summary:
This is a continuation of the mini project on Deep Learning with the Adult income dataset, focusing on hyper parameter tuning.

Have the following:
 *  ColumnTransformer with OneHotEncoder for categorical features and MinMaxScaler for continuous features.
 *  function create_model(input_dim, num_layers=2, num_neurons=8, act_function='relu'), that returns a model to use
 *  function create_pipeline(m_input_dim, m_nl, m_nn, m_af), that creates a pipeline to preprocess with the ColumnTransformer followed by create_model
 *  function kpipe_fit_pred_score(n_ep, n_bsize, num_pipe_feat, m_nl, m_nn, m_af), that uses the pipeline to create the model, fits training data to given epochs and batch_sizes, predicts on the test data and returns the roc_auc_score and time to do the fit

With these
 *  First tried to ColumnTransform X followed by create_model for a **basic model with num_layers=1, num_neurons=8, act_function='relu'; then fit with epochs=10 and batch_size=64. roc_auc_score with basic_model: 0.894**
 *  Created lists with different combinations of [n_epochs, n_batchsize, m_nlayers, m_nunits, m_activationfunctions]. With this, **best roc_auc_score is 0.7926 with**
    **epochs=32  batch_size=64 nlayers=3  nunits=8  actfunc=relu score=0.7926 **
    **epochs=40  batch_size=64 nlayers=3  nunits=8  actfunc=sigmoid score=0.7926 **

**With the pipeline,** </br>
**epochs=10  batch_size=64 nlayers=1  nunits=8  actfunc=relu score=0.7638** </br>
Without the pipeline, where the ColumnTransform is on the entire X (not Xtrain and Xtest separately), the basic model gives a much better roc_auc_score of 0.894 for the same set of parameters.

